In [ ]:
from __future__ import division, print_function, unicode_literals

import numpy as np
import os, glob
import json
import matplotlib.pyplot as plt
import libstempo as t2
from enterprise.signals import utils

import dr2lite_utils as dr2u

%matplotlib inline

In [ ]:
DR2dir = os.path.abspath('/Users/ptb/Projects/pulsar/data/DR2')
datadir = os.path.join(DR2dir, 'release/VersionA')

lite_dir = os.path.join(DR2dir, 'DR2lite_partim')  # cleaned/combined par/tim files
os.system('mkdir -p {}'.format(lite_dir));

# 1) clean up `.par` and `.tim` files

## make list of pulsar names for later

In [ ]:
psrlist = []
psrs = glob.glob(datadir + '/J*')
for psr in psrs:
    name = psr.split('/')[-1]
    psrlist.append(name)
psrlist.sort()

## clean `.par` files

In [ ]:
parfiles = glob.glob(datadir + '/J*/*IPTADR2.par')

for p in parfiles:
    name = p.split('/')[-2]
    outfile = os.path.join(lite_dir, '{}.par'.format(name))
    dr2u.clean_par(p, outfile)

## combine `.tim` files

In [ ]:
timfiles = glob.glob(datadir + '/J*/*IPTADR2.tim')

for t in timfiles:
    name = t.split('/')[-2]
    outfile = os.path.join(lite_dir, '{}.tim'.format(name))
    dr2u.combine_tim(t, outfile)

# 2) Figure of Merit Calculation

Calculate the FoM for each pulsar for each PTA that observes it.

From SNR scaling laws...

$$ \mathrm{FoM} = \frac{T^{13/3}}{\left<{\sigma_{TOA}}\right>^2 \, \Delta t} $$

In [ ]:
def compute_FoM(Tobs, dt, TOAerr):
    """GWB FoM from SNR scaling laws"""
    return Tobs**(13/3) / TOAerr**2 / dt 

In [ ]:
# filter parameters
BW = 1.1
DM_window = 10

Tmin = 5.0  # yrs (of multi-freq coverage)
#sigmax = 5.0  # us

ptas = ['PPTA', 'EPTA', 'NANOGrav']
backends = {'NANOGrav': 
                ['327_ASP', '430_ASP', 'L-wide_ASP', 'S-wide_ASP',
                 '327_PUPPI', '430_PUPPI', 'L-wide_PUPPI',  'S-wide_PUPPI',
                 'Rcvr_800_GASP', 'Rcvr1_2_GASP',
                 'Rcvr_800_GUPPI', 'Rcvr1_2_GUPPI',
                ],
            'PPTA': 
                ['PDFB_10CM', 'PDFB_20CM', 'PDFB_40CM',
                 'CPSR2_20CM', 'CPSR2_50CM',
                 'WBCORR_10CM', 'WBCORR_20CM',
                ],
                
            'EPTA': 
                ['EFF.EBPP.1360', 'EFF.EBPP.1410', 'EFF.EBPP.2639',
                 'JBO.DFB.1400', 'JBO.DFB.1520',
                 'NRT.BON.1400', 'NRT.BON.1600', 'NRT.BON.2000',
                 'WSRT.P1.328', 'WSRT.P1.328.C',
                 'WSRT.P1.382', 'WSRT.P1.382.C',
                 'WSRT.P1.840', 'WSRT.P1.840.C',
                 'WSRT.P1.1380', 'WSRT.P1.1380.C',
                 'WSRT.P1.1380.1',
                 'WSRT.P1.1380.2', 'WSRT.P1.1380.2.C',
                 'WSRT.P1.2273.C',
                ]
           }  # backends to use for each PTA, values for `-group` flag

### Construct `FoM_dict` (takes a long time, skip if already saved)

Runs `filter_psr()` for each pulsar for each PTA, computing the FoM.

In [ ]:
redo = True

dict_file = 'FoM_gwb_dict.json'  # where to save FoM dictionary
data_file = 'FoM_gwb.dat'  # plain text file of FoM data

In [ ]:
if redo or not os.path.isfile(dict_file):
    FoM_dict = {}
    fout = open(data_file, 'w')
    fout.write('{:11}{:9}{:9}{:9}{:9}{:9}{:12}\n'
               .format('Pulsar', 'PTA', 'raw_Tobs', 'mf_Tobs', 'TOAerr', 'cadence', 'GWB_FoM'))

    for psrName in psrlist:
        FoM_dict[psrName] = {}
    
        parfile = os.path.join(lite_dir, '{}.par'.format(psrName))
        timfile = os.path.join(lite_dir, '{}.tim'.format(psrName))
        psr = t2.tempopulsar(parfile, timfile, maxobs=30000)
        for pta in ptas:
            if pta in psr.flagvals('pta'):
                FoM_dict[psrName][pta] = {}
                filt = {'pta':[pta], 'group':backends[pta]}
                psr = dr2u.filter_psr(psr, bw=BW, dt=DM_window, filter_dict=filt, plot=False)
                
                idx = ~psr.deletedmask()
                Tobs_raw = (psr.toas().max()-psr.toas().min()) / 365.25
                if len(psr.toas()[idx]) > 0:
                    U, _ = utils.create_quantization_matrix(psr.toas()[idx]*86400, dt=86400, nmin=1)
                    cadence = (psr.toas()[idx].max() - psr.toas()[idx].min()) / U.shape[1] # mean dt
                    Tobs = (psr.toas()[idx].max()-psr.toas()[idx].min()) / 365.25
                    sigma = 1 / np.mean(1/psr.toaerrs[idx])  # harmonic mean TOAerr
                    FoM = compute_FoM(Tobs, cadence, sigma)
                else:
                    Tobs = 0
                    sigma = np.inf
                    cadence = np.inf
                    FoM = 0
                fout.write('{:11}{:9}{:9.2f}{:9.2f}{:9.2f}{:9.2f}{:12.2f}\n'
                            .format(psrName, pta, Tobs_raw, Tobs, sigma, cadence, FoM))
                FoM_dict[psrName][pta]['Tobs'] = Tobs
                FoM_dict[psrName][pta]['dt'] = cadence
                FoM_dict[psrName][pta]['sigma'] = sigma
                FoM_dict[psrName][pta]['FoM'] = FoM
        del psr
    fout.close()

### save `FoM_dict` as `json` database for later usage

In [ ]:
if redo or not os.path.isfile(dict_file):

    # convert FoM_dict to python float (json can't handle numpy.float128)
    for psr in FoM_dict.keys():
        for pta in FoM_dict[psr].keys():
            for key in FoM_dict[psr][pta].keys():
                FoM_dict[psr][pta][key] = float(FoM_dict[psr][pta][key])

    with open(dict_file, 'w') as fout:
        json.dump(FoM_dict, fout, sort_keys=True, indent=4, separators=(',', ': '))

# 3) Select pulsars to use

Use only PSRs with `Tmin` yrs of multi-frequency data.  Pick one PTA per pulsar.

In [ ]:
dict_file = 'FoM_gwb_dict.json'  # FoM dictionary
with open(dict_file, 'r') as fin:
    FoM_dict = json.load(fin)

In [ ]:
psrdict = {}

for psrname in psrlist:
    best = (None, 0, 0)
    for pta in ptas:
        try:
            this = FoM_dict[psrname][pta]
            if this['Tobs'] > Tmin and this['FoM'] > best[-1]:
                best = (pta, this['Tobs'], this['FoM'])
        except KeyError:
            pass
    if best[0] is not None:
        psrdict.update({psrname: {'pta':[best[0]], 'group':backends[best[0]]}})
        print('{}: {:8}, T = {:5.2f} yr, FoM = {:6.2f}'.format(psrname, *best))

In [ ]:
E,N,P = 0,0,0
for val in psrdict.values():
    val = val['pta']
    if 'EPTA' in val:
        E += 1
    elif 'NANOGrav' in val:
        N += 1
    elif 'PPTA' in val:
        P += 1
print(' EPTA - {:2d}\n   NG - {:2d}\n PPTA - {:2d}'.format(E,N,P))

In [ ]:
# save `psrdict` to file
fname = 'psrdicts/FoM_gwb_psrs.json'
with open(fname, 'w') as fout:
    json.dump(psrdict, fout, sort_keys=True, indent=4, separators=(',', ': '))

# 3) Create filtered par and tim files for DR2-lite GWB analysis

In [ ]:
rundir = 'partim_gwb'

dr2u.make_dataset(psrdict, indir=lite_dir, outdir=rundir,
                  bw=BW, dt=DM_window, plot=False)